In [9]:
import json
from PIL import Image
from transformers import BlipProcessor, BlipForQuestionAnswering
import torch
from tqdm import tqdm
import csv
import warnings
import requests 
warnings.filterwarnings("ignore")

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
reader = csv.reader(open('/kaggle/input/visionpulse-iiitd/IIIT_Dataset.csv', 'r'))
dataset = list(reader)
header = dataset[0]
dataset = dataset[1:]
header

['Image_id', 'Question', 'Answer_1', 'Answer_2', 'Answer_3']

In [6]:
reader = csv.reader(open('/kaggle/input/visionpulse-iiitd/urls.csv', 'r'))
urls_ = list(reader)[1:]
urls = {}
for url in urls_:
    urls[url[0]] = url[1]

In [11]:
data, images= [], []
for i in tqdm(range(len(dataset))):
    data.append({
        'question': dataset[i][1],
        'answers': [dataset[i][2], dataset[i][3], dataset[i][4]],
    })
    content = requests.get(urls[dataset[i][0]]).content 
    f = open('img.jpg','wb') 
    f.write(content) 
    f.close() 
    images.append(Image.open('img.jpg').convert('RGB'))

100%|██████████| 24/24 [00:08<00:00,  2.89it/s]


In [15]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to(device)

preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

In [17]:
model.eval()
with torch.no_grad():
    for i in tqdm(range(len(data))):
        question = data[i]['question']
        raw_image = images[i]
        inputs = processor(raw_image, question, return_tensors="pt").to(device)
        out = model.generate(**inputs)
        answer = processor.decode(out[0], skip_special_tokens=True)
        data[i]['prediction'] = answer

100%|██████████| 24/24 [00:04<00:00,  5.43it/s]


In [19]:
json.dump(data, open(f'BLIP_IIITD.json', 'w'), indent=4)